In [ ]:
from llm_foundation.basic_structs import Provider, LMConfig
from llm_foundation.extractors import WebPageContentExtractor, CitedDocuments
from llm_foundation.lm import get_model_catalog, get_lm
from llm_foundation.routing import ToolMaster

from langchain.prompts import ChatPromptTemplate

In [ ]:
get_model_catalog(Provider.LC, ["gpt-4o-mini"])

In [ ]:
lm_config = LMConfig(model="gpt-4o-mini", provider=Provider.LC)
lm = get_lm(lm_config)

In [ ]:
web_doc = "https://lilianweng.github.io/posts/2023-06-23-agent/"

webpage_extractor_tool = WebPageContentExtractor.build_tool(lm, CitedDocuments)

print(type(webpage_extractor_tool))
print(webpage_extractor_tool.name)
print(webpage_extractor_tool.description)
print(webpage_extractor_tool.args)

In [ ]:
tools = [webpage_extractor_tool]
query = "calculate 12 * 4"
query_requiring_tools = f"extract cited documents from {web_doc}"

print(lm.invoke(query))

lm_with_tools = lm.bind_tools(tools)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | lm_with_tools # | ToolsAgentOutputParser()
chain_response = chain.invoke({"input": query_requiring_tools})
print(type(chain_response))
print(chain_response)

# Tool Master

In [ ]:
tool_master = ToolMaster(tools)

In [ ]:
tool_response = tool_master.call_tools(chain_response)
tool_response